In [6]:
import pandas as pd
import ast

In [8]:
df = pd.read_csv(r'C:\Users\nahue\Searches\Escritorio\CARRERA HENRY\PPI\Movies\credits_transf.csv',low_memory=False)

In [9]:
df.head(2)

,id,cast_filtrado,crew_filtrado
0,862,"[{'name': 'Tom Hanks'}, {'name': 'Tim Allen'},...","[{'job': 'Director', 'name': 'John Lasseter'}]"
1,8844,"[{'name': 'Robin Williams'}, {'name': 'Jonatha...","[{'job': 'Director', 'name': 'Joe Johnston'}]"


In [10]:
df.shape

(45476, 3)

In [11]:
#Al querer probar funciones, me sale un error de que el Kernel se muere. Posiblemente porque el archivo es muy grande. Lo achico.

# Selecciono las primeras 2000 filas
df_credits_reducido = df.head(2000)

# Guardo el nuevo archivo CSV
df_credits_reducido.to_csv('credits_2000.csv', index=False)

In [12]:
df1 = pd.read_csv(r'C:\Users\nahue\Searches\Escritorio\CARRERA HENRY\PPI\Movies\credits_2000.csv', low_memory=False)

In [13]:
df1.shape

(2000, 3)

In [14]:
# Convierto la columna 'cast_filtrado' de cadenas de texto a listas de diccionarios
df1['cast_filtrado'] = df1['cast_filtrado'].apply(ast.literal_eval)


# Función para desglosar los diccionarios en filas individuales para 'cast_filtrado'
def desglosar_cast(row):
    return pd.DataFrame(row['cast_filtrado'])

# Creo el df para actores
actores_list = [] # Inicializamos una lista vacía para almacenar los datos de los actores.
for index, row in df1.iterrows(): # Iteramos sobre cada fila del DataFrame.
    if isinstance(row['cast_filtrado'], list):
        for actor in row['cast_filtrado']: # Verificamos si la columna cast_filtrado contiene una lista.
            actores_list.append({'id': row['id'], 'actor': actor['name']})# Añadimos un nuevo diccionario a actores_list con el id de la película y el name del actor.

df_actores = pd.DataFrame(actores_list)

# Veo el df resultante de actores
print("DataFrame de Actores:")
print(df_actores[['id', 'actor']])


DataFrame de Actores:
         id            actor
0       862        Tom Hanks
1       862        Tim Allen
2       862      Don Rickles
3       862       Jim Varney
4       862    Wallace Shawn
...     ...              ...
35209  2107  Woody Harrelson
35210  2107     Rick Moranis
35211  2107   Robert Picardo
35212  2107  George Plimpton
35213  2107    Andrew Amador

[35214 rows x 2 columns]


In [15]:
df_actores.to_csv('actores.csv', index=False)

In [16]:
# Convierto la columna 'crew_filtrado' de cadenas de texto a listas de diccionarios
df1['crew_filtrado'] = df1['crew_filtrado'].apply(ast.literal_eval)

# Función para desglosar los diccionarios en filas individuales para 'crew_filtrado'
def desglosar_crew(row):
    return pd.DataFrame(row['crew_filtrado'])

# Creo el df para directores
directores_list = []
for index, row in df1.iterrows():
    if isinstance(row['crew_filtrado'], list):
        for director in row['crew_filtrado']:
            directores_list.append({'id': row['id'], 'director': director['name'], 'job': director['job']})

df_directores = pd.DataFrame(directores_list)

# Vo el DataFrame resultante de directores
print("DataFrame de Directores:")
print(df_directores[['id', 'director', 'job']])

DataFrame de Directores:
         id          director       job
0       862     John Lasseter  Director
1      8844      Joe Johnston  Director
2     15602     Howard Deutch  Director
3     31357   Forest Whitaker  Director
4     11862     Charles Shyer  Director
...     ...               ...       ...
2114  27332        Tim Hunter  Director
2115     97  Steven Lisberger  Director
2116  10419     Thomas Carter  Director
2117  11675       Steve Miner  Director
2118   2107      Mick Jackson  Director

[2119 rows x 3 columns]


In [17]:
df_directores = df_directores.drop(['job'], axis=1)

In [18]:
df_directores.to_csv('directores.csv', index=False)

In [19]:
# Uno los DataFrames por la columna 'id' usando left join
df_unido = pd.merge(df_actores, df_directores, on='id', how='left')

# Veo el DataFrame resultante
print("DataFrame Unificado con left join:")
df_unido

DataFrame Unificado con left join:


,id,actor,director
0,862,Tom Hanks,John Lasseter
1,862,Tim Allen,John Lasseter
2,862,Don Rickles,John Lasseter
3,862,Jim Varney,John Lasseter
4,862,Wallace Shawn,John Lasseter
...,...,...,...
36991,2107,Woody Harrelson,Mick Jackson
36992,2107,Rick Moranis,Mick Jackson
36993,2107,Robert Picardo,Mick Jackson
36994,2107,George Plimpton,Mick Jackson


In [20]:
#Busco actores que son directores:

# Añado una columna que indique si el actor es también el director. Si los valores coinciden sera TRUE, sino FALSE.
df_unido['actor_y_director'] = df_unido['actor'] == df_unido['director']

# Filtrar las filas donde haya coincidencia
df_coincidencias = df_unido[df_unido['actor_y_director']]

# Veo el DataFrame resultante con coincidencias
print("DataFrame con coincidencias de actor y director:")
print(df_coincidencias)

DataFrame con coincidencias de actor y director:
          id              actor           director  actor_y_director
242    12110         Mel Brooks         Mel Brooks              True
419        5  Quentin Tarantino  Quentin Tarantino              True
681      902          Marc Caro          Marc Caro              True
1140   11448        Woody Allen        Woody Allen              True
1314    4482    Josiane Balasko    Josiane Balasko              True
...      ...                ...                ...               ...
34190  10131       Renny Harlin       Renny Harlin              True
34415    948     John Carpenter     John Carpenter              True
34471  10676  Tommy Lee Wallace  Tommy Lee Wallace              True
36082   1779      Michael Moore      Michael Moore              True
36921  53565        Walt Disney        Walt Disney              True

[201 rows x 4 columns]


In [21]:
#Elimino las coincidencias para realizar la funcion get_actor ya que no debe considerar directores.

# Filtro las filas donde 'actor y director' es False
df_sin_coincidencias = df_unido[df_unido['actor_y_director'] == False]

# Ver el DataFrame resultante sin coincidencias
print("DataFrame sin coincidencias de actor y director:")
print(df_sin_coincidencias)

DataFrame sin coincidencias de actor y director:
         id            actor       director  actor_y_director
0       862        Tom Hanks  John Lasseter             False
1       862        Tim Allen  John Lasseter             False
2       862      Don Rickles  John Lasseter             False
3       862       Jim Varney  John Lasseter             False
4       862    Wallace Shawn  John Lasseter             False
...     ...              ...            ...               ...
36991  2107  Woody Harrelson   Mick Jackson             False
36992  2107     Rick Moranis   Mick Jackson             False
36993  2107   Robert Picardo   Mick Jackson             False
36994  2107  George Plimpton   Mick Jackson             False
36995  2107    Andrew Amador   Mick Jackson             False

[36795 rows x 4 columns]


In [22]:
#Elimino la columna actor_y_director y guardo el df aparte.
df_sin_coincidencias = df_sin_coincidencias.drop(columns=['actor_y_director'])

In [23]:
df_sin_coincidencias.to_csv('actores_no_directores.csv')

In [25]:
df4 = pd.read_csv(r'C:\Users\nahue\Searches\Escritorio\CARRERA HENRY\PPI\Movies\movies_dataset_transf.csv',low_memory=False)

In [26]:
df4.head(2)

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,...,spoken_languages,status,tagline,title,vote_average,vote_count,id_movie,movie_name,release_year,return
0,30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,10194,Toy Story Collection,1995.0,12.451801
1,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,0,NaN,1995.0,4.043035


In [27]:
# Selecciono las columnas que necesito para la funcion get_actor
columnas_necesarias = ['id', 'title', 'return']
df_necesario = df4[columnas_necesarias]

# Ver el DataFrame resultante
df_necesario

,id,title,return
0,862,Toy Story,12.451801
1,8844,Jumanji,4.043035
2,15602,Grumpier Old Men,0.000000
3,31357,Waiting to Exhale,5.090760
4,11862,Father of the Bride Part II,0.000000
...,...,...,...
1995,10419,Swing Kids,0.000000
1996,11675,Halloween: H20,3.237749
1997,2107,L.A. Story,0.000000
1998,6471,The Jerk,0.000000


In [28]:
df_necesario.to_csv('df_para_get_actor.csv')

In [29]:
# Uno los DataFrames por la columna 'id' usando left join
df_get_actor = pd.merge(df_sin_coincidencias, df_necesario, on='id', how='left')

# Veo el DataFrame resultante
df_get_actor

,id,actor,director,title,return
0,862,Tom Hanks,John Lasseter,Toy Story,12.451801
1,862,Tim Allen,John Lasseter,Toy Story,12.451801
2,862,Don Rickles,John Lasseter,Toy Story,12.451801
3,862,Jim Varney,John Lasseter,Toy Story,12.451801
4,862,Wallace Shawn,John Lasseter,Toy Story,12.451801
...,...,...,...,...,...
36826,2107,Woody Harrelson,Mick Jackson,L.A. Story,0.000000
36827,2107,Rick Moranis,Mick Jackson,L.A. Story,0.000000
36828,2107,Robert Picardo,Mick Jackson,L.A. Story,0.000000
36829,2107,George Plimpton,Mick Jackson,L.A. Story,0.000000


In [30]:
df_get_actor = df_get_actor.drop(columns=['director'])

In [31]:
df_get_actor.head(2)

,id,actor,title,return
0,862,Tom Hanks,Toy Story,12.451801
1,862,Tim Allen,Toy Story,12.451801


In [32]:
df_get_actor.to_csv('df_para_funcion_get_actor.csv')

### Obtener actor, su éxito a través del retorno, cantidad de películas en las que participó y su retorno promedio

In [33]:
def get_actor(nombre_actor):
    # Filtrar el DataFrame para obtener solo las filas del actor especificado ignorando mayúsculas y minúsculas
    df_actor = df_get_actor[df_get_actor['actor'].str.lower() == nombre_actor.lower()]
    
    # Calcular la cantidad de películas en las que ha participado
    cantidad_peliculas = df_actor.shape[0] 
    # con shape[0] obtengo el número de filas en df_actor, que corresponde a la cantidad de películas en las que el actor ha participado.
    
    # Calcular el retorno total
    retorno_total = df_actor['return'].sum()
    
    # Calcular el promedio de retorno por filmación
    promedio_retorno = df_actor['return'].mean()
    
    # Formatear el resultado
    resultado = (
        f"El actor {nombre_actor} ha participado de {cantidad_peliculas} cantidad de filmaciones, "
        f"el mismo ha conseguido un retorno de {retorno_total} con un promedio de {promedio_retorno:.2f} por filmación"
    )
    
    return resultado

In [34]:
# Ejemplo de uso de la función
nombre_actor = 'Frank Welker'
resultado = get_actor(nombre_actor)
print(resultado)

El actor Frank Welker ha participado de 33 cantidad de filmaciones, el mismo ha conseguido un retorno de 158.51834432334533 con un promedio de 4.80 por filmación


In [35]:
#Verifico que los datos son correctos:
cantidad = df_get_actor['actor'].value_counts()
print(cantidad)

actor
Frank Welker         33
Sterling Holloway    26
Samuel L. Jackson    25
Whoopi Goldberg      24
Jim Cummings         22
                     ..
Helen Spring          1
Lloyd Williams        1
Paula Winslowe        1
Wilson Wood           1
Victoria Tennant      1
Name: count, Length: 21328, dtype: int64


### Obtener director, su éxito a través del retorno, nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma

In [36]:
# Selecciono las columnas que necesito para la funcion get_director
columnas_necesarias2 = ['id', 'title','release_date','budget','revenue','return']
df_necesario2 = df4[columnas_necesarias2]

# Ver el DataFrame resultante
df_necesario2.head(2)

,id,title,release_date,budget,revenue,return
0,862,Toy Story,1995-10-30,30000000.0,373554033.0,12.451801
1,8844,Jumanji,1995-12-15,65000000.0,262797249.0,4.043035


In [37]:
df_directores.head(2)

,id,director
0,862,John Lasseter
1,8844,Joe Johnston


In [38]:
# Uno los DataFrames por la columna 'id' usando left join
df_get_director = pd.merge(df_directores, df_necesario2, on='id', how='left')

# Veo el DataFrame resultante
df_get_director.head(3)

,id,director,title,release_date,budget,revenue,return
0,862,John Lasseter,Toy Story,1995-10-30,30000000.0,373554033.0,12.451801
1,8844,Joe Johnston,Jumanji,1995-12-15,65000000.0,262797249.0,4.043035
2,15602,Howard Deutch,Grumpier Old Men,1995-12-22,0.0,0.0,0.000000


In [39]:
df_necesario2.to_csv('df_para_get_director.csv')

In [42]:
df_get_director.to_csv('df_para_funcion_get_director.csv')

In [43]:
def get_director(nombre_director):
    # Filtro el df para obtener solo las filas del director especificado
    df_director = df_get_director[df_get_director['director'].str.lower() == nombre_director.lower()]
    
    # Calculo el retorno total
    retorno_total = df_director['return'].sum()
    
    # Creo la lista de películas con la información solicitada
    peliculas_info = [] #Iteramos sobre las filas del df filtrado para extraer la información de cada película y crear una lista de diccionarios peliculas_info.
    for index, row in df_director.iterrows():
        pelicula = {
            'title': row['title'],
            'release_date': row['release_date'],
            'return': row['return'],
            'budget': row['budget'],
            'revenue': row['revenue']
        }
        peliculas_info.append(pelicula)
    
    # Formatear la información de las películas
    peliculas_str = "\n".join([f"Película: {p['title']}, Fecha de lanzamiento: {p['release_date']}, Retorno: {p['return']}, Costo: {p['budget']}, Ganancia: {p['revenue']}" for p in peliculas_info])
    
    # Formatear el resultado
    resultado = (
        f"El director {nombre_director} ha conseguido un retorno total de {retorno_total}.\n"
        f"Ha dirigido las siguientes películas:\n{peliculas_str}"
    )
    
    return resultado

In [44]:
# Chequeo de la función
nombre_director = 'Joe Johnston'
resultado = get_director(nombre_director)
print(resultado)

El director Joe Johnston ha conseguido un retorno total de 12.985677228968255.
Ha dirigido las siguientes películas:
Película: Jumanji, Fecha de lanzamiento: 1995-12-15, Retorno: 4.0430346, Costo: 65000000.0, Ganancia: 262797249.0
Película: The Pagemaster, Fecha de lanzamiento: 1994-11-23, Retorno: 0.5063217777777778, Costo: 27000000.0, Ganancia: 13670688.0
Película: Honey, I Shrunk the Kids, Fecha de lanzamiento: 1989-06-22, Retorno: 6.960130375, Costo: 32000000.0, Ganancia: 222724172.0
Película: The Rocketeer, Fecha de lanzamiento: 1991-06-21, Retorno: 1.4761904761904765, Costo: 42000000.0, Ganancia: 62000000.0
